In [1]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS
from flightsql import FlightSQLClient

from datetime import datetime

In [3]:
token = "whl_f4m7pZbnLdO6KHYmNFjFdJaGimywqZXMezcOCwFcwJyUOW0nomnbHXzMdrxf3TeKOGbzpUW4B2rDXgUu5Q=="
org = "csse4011"
url = "http://localhost:8086"

bucket="Demo_data"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org, debug=False)

write_api = write_client.write_api(write_options=SYNCHRONOUS)

In [4]:

point = (
    Point("Position")
    .field("X", 0)
    .field("Y", 0)
    .field("Z", 0)
    .field("Velocity", 2)
    )
write_api.write(bucket=bucket, org="csse4011", record=point)
                        